In [4]:
from __future__ import print_function
import findspark
findspark.init()
findspark.find()
import pyspark
findspark.find()
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

In [5]:
if __name__ == "__main__":
    spark = SparkSession\
    .builder\
    .appName("Linearregwithspark")\
    .getOrCreate()

In [6]:
dataset = spark.read.csv("Admission_Prediction.csv",header=True)
dataset.show()

+---------+-----------+-----------------+----+----+----+--------+---------------+
|GRE Score|TOEFL Score|University Rating| SOP| LOR|CGPA|Research|Chance of Admit|
+---------+-----------+-----------------+----+----+----+--------+---------------+
|   337.00|     118.00|                4|4.50|4.50|9.65|    1.00|           0.92|
|   324.00|     107.00|                4|4.00|4.50|8.87|    1.00|           0.76|
|     null|     104.00|                3|3.00|3.50|8.00|    1.00|           0.72|
|   322.00|     110.00|                3|3.50|2.50|8.67|    1.00|           0.80|
|   314.00|     103.00|                2|2.00|3.00|8.21|    0.00|           0.65|
|   330.00|     115.00|                5|4.50|3.00|9.34|    1.00|           0.90|
|   321.00|     109.00|             null|3.00|4.00|8.20|    1.00|           0.75|
|   308.00|     101.00|                2|3.00|4.00|7.90|    0.00|           0.68|
|   302.00|     102.00|                1|2.00|1.50|8.00|    0.00|           0.50|
|   323.00|     

In [7]:
type(dataset)

pyspark.sql.dataframe.DataFrame

In [8]:
dataset.printSchema()

root
 |-- GRE Score: string (nullable = true)
 |-- TOEFL Score: string (nullable = true)
 |-- University Rating: string (nullable = true)
 |-- SOP: string (nullable = true)
 |-- LOR: string (nullable = true)
 |-- CGPA: string (nullable = true)
 |-- Research: string (nullable = true)
 |-- Chance of Admit: string (nullable = true)



In [15]:
from pyspark.sql.functions import col
new_data = dataset.select(*(col(c).cast('float').alias(c) for c in dataset.columns))

In [16]:
(col(c).cast('float').alias(c) for c in dataset.columns)

<generator object <genexpr> at 0x000001C5D2E1BA40>

In [17]:
list((col(c).cast('float').alias(c) for c in dataset.columns))

[Column<b'CAST(GRE Score AS FLOAT) AS `GRE Score`'>,
 Column<b'CAST(TOEFL Score AS FLOAT) AS `TOEFL Score`'>,
 Column<b'CAST(University Rating AS FLOAT) AS `University Rating`'>,
 Column<b'CAST(SOP AS FLOAT) AS `SOP`'>,
 Column<b'CAST(LOR AS FLOAT) AS `LOR`'>,
 Column<b'CAST(CGPA AS FLOAT) AS `CGPA`'>,
 Column<b'CAST(Research AS FLOAT) AS `Research`'>,
 Column<b'CAST(Chance of Admit AS FLOAT) AS `Chance of Admit`'>]

In [18]:
new_data.printSchema()

root
 |-- GRE Score: float (nullable = true)
 |-- TOEFL Score: float (nullable = true)
 |-- University Rating: float (nullable = true)
 |-- SOP: float (nullable = true)
 |-- LOR: float (nullable = true)
 |-- CGPA: float (nullable = true)
 |-- Research: float (nullable = true)
 |-- Chance of Admit: float (nullable = true)



In [19]:
from pyspark.sql.functions import col, when, isnan, count

In [24]:
new_data.select([count(when(col(c).isNull(),c)).alias(c) for c in dataset.columns]).show() #finding nulls

+---------+-----------+-----------------+---+---+----+--------+---------------+
|GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|
+---------+-----------+-----------------+---+---+----+--------+---------------+
|       15|         10|               15|  0|  0|   0|       0|              0|
+---------+-----------+-----------------+---+---+----+--------+---------------+



In [25]:
from pyspark.ml.feature import Imputer

In [26]:
imputer = Imputer(inputCols=["GRE Score", "TOEFL Score","University Rating"], 
                  outputCols=["GRE Score", "TOEFL Score","University Rating"])
model = imputer.fit(new_data)

imputed_data = model.transform(new_data)

In [27]:
imputed_data.select([count(when(col(c).isNull(),c)).alias(c) for c in dataset.columns]).show() #finding nulls

+---------+-----------+-----------------+---+---+----+--------+---------------+
|GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|
+---------+-----------+-----------------+---+---+----+--------+---------------+
|        0|          0|                0|  0|  0|   0|       0|              0|
+---------+-----------+-----------------+---+---+----+--------+---------------+



In [28]:
features = imputed_data.drop('Chance of Admit')

In [30]:
# assemblin

assembler = VectorAssembler(inputCols= features.columns ,outputCol= 'features')

In [31]:
output = assembler.transform(imputed_data)
output.show()

+---------+-----------+-----------------+---+---+----+--------+---------------+--------------------+
|GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|            features|
+---------+-----------+-----------------+---+---+----+--------+---------------+--------------------+
|    337.0|      118.0|              4.0|4.5|4.5|9.65|     1.0|           0.92|[337.0,118.0,4.0,...|
|    324.0|      107.0|              4.0|4.0|4.5|8.87|     1.0|           0.76|[324.0,107.0,4.0,...|
|316.55878|      104.0|              3.0|3.0|3.5| 8.0|     1.0|           0.72|[316.558776855468...|
|    322.0|      110.0|              3.0|3.5|2.5|8.67|     1.0|            0.8|[322.0,110.0,3.0,...|
|    314.0|      103.0|              2.0|2.0|3.0|8.21|     0.0|           0.65|[314.0,103.0,2.0,...|
|    330.0|      115.0|              5.0|4.5|3.0|9.34|     1.0|            0.9|[330.0,115.0,5.0,...|
|    321.0|      109.0|        3.1216495|3.0|4.0| 8.2|     1.0|           0.75|[321.0,109.0

In [33]:
output = output.select('features','Chance of Admit')
output.show()

+--------------------+---------------+
|            features|Chance of Admit|
+--------------------+---------------+
|[337.0,118.0,4.0,...|           0.92|
|[324.0,107.0,4.0,...|           0.76|
|[316.558776855468...|           0.72|
|[322.0,110.0,3.0,...|            0.8|
|[314.0,103.0,2.0,...|           0.65|
|[330.0,115.0,5.0,...|            0.9|
|[321.0,109.0,3.12...|           0.75|
|[308.0,101.0,2.0,...|           0.68|
|[302.0,102.0,1.0,...|            0.5|
|[323.0,108.0,3.0,...|           0.45|
|[325.0,106.0,3.0,...|           0.52|
|[327.0,111.0,4.0,...|           0.84|
|[316.558776855468...|           0.78|
|[307.0,109.0,3.0,...|           0.62|
|[311.0,104.0,3.0,...|           0.61|
|[314.0,105.0,3.0,...|           0.54|
|[317.0,107.0,3.0,...|           0.66|
|[319.0,106.0,3.0,...|           0.65|
|[318.0,110.0,3.0,...|           0.63|
|[303.0,102.0,3.0,...|           0.62|
+--------------------+---------------+
only showing top 20 rows



In [34]:
train_ds, test_ds = output.randomSplit([0.7,0.3],seed= 10)

In [36]:
train_ds.show()

+--------------------+---------------+
|            features|Chance of Admit|
+--------------------+---------------+
|[290.0,100.0,1.0,...|           0.47|
|[293.0,97.0,2.0,2...|           0.64|
|[294.0,95.0,1.0,1...|           0.49|
|[295.0,93.0,1.0,2...|           0.46|
|[295.0,101.0,2.0,...|           0.69|
|[296.0,97.0,2.0,1...|           0.49|
|[296.0,99.0,2.0,2...|           0.61|
|[296.0,101.0,1.0,...|            0.6|
|[297.0,96.0,2.0,2...|           0.43|
|[297.0,98.0,2.0,2...|           0.59|
|[297.0,99.0,4.0,3...|           0.54|
|[297.0,100.0,1.0,...|           0.52|
|[297.0,101.0,3.0,...|           0.57|
|[298.0,92.0,1.0,2...|           0.51|
|[298.0,97.0,3.121...|           0.45|
|[298.0,98.0,2.0,4...|           0.34|
|[298.0,99.0,1.0,1...|           0.53|
|[298.0,100.0,3.0,...|           0.58|
|[298.0,101.0,2.0,...|           0.54|
|[298.0,101.0,4.0,...|           0.53|
+--------------------+---------------+
only showing top 20 rows



In [37]:
test_ds.show()

+--------------------+---------------+
|            features|Chance of Admit|
+--------------------+---------------+
|[290.0,104.0,4.0,...|           0.45|
|[294.0,93.0,1.0,1...|           0.46|
|[295.0,96.0,2.0,1...|           0.47|
|[295.0,99.0,1.0,2...|           0.37|
|[295.0,99.0,2.0,2...|           0.57|
|[296.0,95.0,2.0,3...|           0.44|
|[296.0,99.0,2.0,3...|           0.47|
|[297.0,96.0,2.0,2...|           0.34|
|[298.0,98.0,2.0,1...|           0.44|
|[298.0,105.0,3.0,...|           0.69|
|[299.0,96.0,2.0,1...|           0.54|
|[299.0,100.0,2.0,...|           0.51|
|[299.0,102.0,3.0,...|           0.56|
|[300.0,95.0,2.0,3...|           0.62|
|[301.0,97.0,2.0,3...|           0.44|
|[301.0,98.0,1.0,2...|           0.67|
|[301.0,99.0,3.0,2...|           0.68|
|[301.0,100.0,3.0,...|           0.67|
|[302.0,99.0,3.0,2...|           0.52|
|[302.0,101.0,2.0,...|           0.46|
+--------------------+---------------+
only showing top 20 rows



In [39]:
lin_reg = LinearRegression( featuresCol='features',labelCol='Chance of Admit')
model = lin_reg.fit(train_ds)

In [42]:
print("Coefficients: " , model.coefficients)
print("Intercept: " , model.intercept)


Coefficients:  [0.0017228904314267883,0.0024196878972895914,0.008363132881043336,-0.003119868738041775,0.019380894683278013,0.11585906448003329,0.03150215129789561]
Intercept:  -1.177368929445545


In [47]:
train_summary =  model.summary
print("RMSE: ", train_summary.rootMeanSquaredError)
print("R2: ",train_summary.r2)

RMSE:  0.06104107962099275
R2:  0.8034333549854024


In [ ]:
predictions = linear_model.transform(test_df)
predictions.select("prediction","Chance of Admit","features").show()